# Capstone project - Toronto Neighborhoods

### This  notebook is for build the code, to scrape and clean up Wikipedia page 

In this notebook I will to explore, segment, and cluster the neighborhoods in the city of Toronto. 

Firs, I let's download all the dependencies that we will need.

In [62]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

I used BeautifulSoup library to scrap the Wikipedia page. Then, I transforme the data in format requiered, cleaning data remove the Borough whit Not assigned and missings, and finally obtein a DataFrame.

In [63]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bs = BeautifulSoup(page,'xml')

In [64]:
table = bs.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [65]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

In [66]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[(~df['PostalCode'].isnull()) & (df['Borough'] != 'Not assigned')]
df = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(list) 
df = pd.DataFrame(df)
df1 = df.sort_values(by=['Neighbourhood'])
df1 = df1.reset_index()

In [81]:
column_names = ['Neighbours'] 
neights = pd.DataFrame(columns=column_names)

prueba = df1.loc[:,'Neighbourhood']

for row in prueba:
    prueba1 = ','.join(row)
    neights = neights.append({'Neighbours' : prueba1}, ignore_index = True)

In [82]:
torn_venues = df1.drop(['Neighbourhood'], axis= 1)
trt_venues = pd.merge(torn_venues, neights, left_index=True, right_index=True)
trt_venues.head()

,PostalCode,Borough,Neighbours
0,M5H,Downtown Toronto,"Adelaide,King,Richmond"
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel..."
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
4,M8W,Etobicoke,"Alderwood,Long Branch"


#### Load and explore the data

Next, let's load the data, latitude and longitude.

In [12]:
df1.shape

(103, 3)

In [13]:
path ='http://cocl.us/Geospatial_data'
dfl = pd.read_csv(path)
dfl.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
dfc = pd.merge(trt_venues, dfl, left_on ='PostalCode', right_on = 'Postal Code')
dfc.head()

,PostalCode,Borough,vecis,Postal Code,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide,King,Richmond",M5H,43.650571,-79.384568
1,M1S,Scarborough,Agincourt,M1S,43.794200,-79.262029
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",M1V,43.815252,-79.284577
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",M9V,43.739416,-79.588437
4,M8W,Etobicoke,"Alderwood,Long Branch",M8W,43.602414,-79.543484


In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfc['Borough'].unique()),
        dfc.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toront City.

In [17]:
address = ' Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of New York with neighborhoods superimposed on top.

In [19]:
# create map of Toront using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neightbourhood in zip(dfc['Latitude'], dfc['Longitude'], dfc['Borough'], dfc['vecis']):
    label = '{}, {}'.format(neightbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

#### Let's explore the first neighborhood in our dataframe.

In [21]:
dfc.loc[0, 'vecis']

'Adelaide,King,Richmond'

In [22]:
neighborhood_latitude = dfc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfc.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfc.loc[0, 'vecis'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Adelaide,King,Richmond are 43.65057120000001, -79.3845675.


In [23]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=ZVQR4LO4HEVNDYY5IG23KBDF2OOA00HWLLOM5FFTX5YHOQRB&client_secret=QR5M3BNEU5UQZGOXYKNEBPZLMPA3O3HC54ISAERZOKAOWCCA&v=20180605&ll=43.65057120000001,-79.3845675&radius=500&limit=100'

Send the GET request and examine the resutls, after to create the GET request URL. Name your URL **url**. 

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7bf84c018cbb002ccc38fc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 128,
  'suggestedBounds': {'ne': {'lat': 43.65507120450001,
    'lng': -79.37835988143604},
   'sw': {'lat': 43.64607119550001, 'lng': -79.39077511856397}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a520e5f720e3',
       'name': 'Four Seasons Centre for the Performing Arts',
       'location': {'address': '145 Queen St. W',
        'crossStreet': 'at University Ave.',
        'lat': 43.650592,
        'lng': -79.38

Let's borrow the **get_category_type** function from the Foursquare.

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
1,The Keg Steakhouse & Bar,Steakhouse,43.649937,-79.384196
2,Nathan Phillips Square,Plaza,43.652270,-79.383516
3,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
4,Shangri-La Toronto,Hotel,43.649129,-79.386557


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['vecis', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
toronto_venues = getNearbyVenues(names = dfc['vecis'],
                                   latitudes = dfc['Latitude'],
                                   longitudes = dfc['Longitude']
                                  )


Adelaide,King,Richmond
Agincourt
Agincourt North,L'Amoreaux East,Milliken,Steeles East
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
Alderwood,Long Branch
Bathurst Manor,Downsview North,Wilson Heights
Bayview Village
Bedford Park,Lawrence Manor East
Berczy Park
Birch Cliff,Cliffside West
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Brockton,Exhibition Place,Parkdale Village
Business Reply Mail Processing Centre 969 Eastern
CFB Toronto,Downsview East
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Cabbagetown,St. James Town
Caledonia-Fairbanks
Canada Post Gateway Processing Centre
Cedarbrae
Central Bay Street
Chinatown,Grange Park,Kensington Market
Christie
Church and Wellesley
Clairlea,Golden Mile,Oakridge
Clarks Corners,Sullivan,Tam O'Shanter
Cliffcrest,Cliffside,Scarborough Village West
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Commerce 

#### Let's check the size of the resulting dataframe

In [34]:
print(toronto_venues.shape)
toronto_venues.tail()

(2260, 7)


,vecis,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2255,Woodbine Heights,43.695344,-79.318389,Beauty Boutique by Shoppers Drug Mart,43.699514,-79.319287,Cosmetics Shop
2256,York Mills West,43.752758,-79.400049,TD Canada Trust Branch and ATM,43.750923,-79.401004,Bank
2257,York Mills West,43.752758,-79.400049,Kitchen Food Fair,43.751298,-79.401393,Convenience Store
2258,York Mills West,43.752758,-79.400049,Tournament Park,43.751257,-79.399717,Park
2259,York Mills West,43.752758,-79.400049,Yorkminster Park,43.755489,-79.401712,Park


Let's check how many venues were returned for each neighborhood

In [35]:
toronto_venues.groupby('vecis').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
vecis,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56


In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


## 3. Analyze Each Neighborhood

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['vecis'] = toronto_venues['vecis'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,vecis,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0,0,0,0

In [38]:
toronto_onehot.shape

(2260, 275)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
toronto_grouped = toronto_onehot.groupby('vecis').mean().reset_index()
toronto_grouped

,vecis,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.00000

In [40]:
toronto_grouped.shape

(100, 275)

#### Let's print each neighborhood along with the top 5 most common venues

In [41]:
num_top_venues = 5

for hood in toronto_grouped['vecis']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['vecis'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2   Steakhouse  0.04
3          Bar  0.04
4        Hotel  0.03


----Agincourt----
                 venue  freq
0       Breakfast Spot  0.25
1               Lounge  0.25
2       Sandwich Place  0.25
3         Skating Rink  0.25
4  Monument / Landmark  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2  Molecular Gastronomy Restaurant   0.0
3                      Music Venue   0.0
4                           Museum   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1            Beer Store  0.11
2  Fast Food Restaurant  0.11
3   Fried Chicken Joint  0.11
4           Coffee Shop  0.11


----Alderwood,Long Branch----
            ve

           venue  freq
0  Grocery Store  0.33
1           Bank  0.17
2          Hotel  0.17
3  Shopping Mall  0.17
4           Park  0.17


----Downsview,North Park,Upwood Park----
                        venue  freq
0              Massage Studio  0.25
1                        Park  0.25
2                      Bakery  0.25
3  Construction & Landscaping  0.25
4                    Pharmacy  0.00


----East Birchmount Park,Ionview,Kennedy Park----
               venue  freq
0     Discount Store  0.29
1  Convenience Store  0.14
2        Bus Station  0.14
3        Coffee Shop  0.14
4   Department Store  0.14


----East Toronto----
                 venue  freq
0                 Park  0.50
1    Convenience Store  0.25
2  Rental Car Location  0.25
3    Accessories Store  0.00
4  Monument / Landmark  0.00


----Emery,Humberlea----
                           venue  freq
0  Paper / Office Supplies Store   0.5
1                 Baseball Field   0.5
2                   Neighborhood   0.0
3         

                             venue  freq
0                Martial Arts Dojo   1.0
1  Molecular Gastronomy Restaurant   0.0
2                      Music Venue   0.0
3                           Museum   0.0
4                    Moving Target   0.0


----St. James Town----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.06
2               Hotel  0.05
3          Restaurant  0.05
4  Italian Restaurant  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.04
2                Café  0.04
3  Seafood Restaurant  0.03
4        Cocktail Bar  0.03


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3  American Restaurant  0.05
4               Bakery  0.05


----The Annex,North Midtown,Yorkville----
                           venue  freq
0                    Coffee Shop  0.13
1                           C

#### Let's put that into a *pandas* dataframe

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['vecis']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['vecis'] = toronto_grouped['vecis']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhood_venues_sorted.head()

,vecis,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Cosmetics Shop,Hotel,American Restaurant,Burger Joint,Restaurant
1,Agincourt,Breakfast Spot,Sandwich Place,Lounge,Skating Rink,Yoga Studio,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Beer Store,Coffee Shop,Pharmacy,Empanada Restaurant,Ethiopian Restaurant
4,"Alderwood,Long Branch",Pizza Place,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pool,Gym,Pharmacy,Empanada Restaurant,Ethiopian Restaurant


In [47]:
toronto_data = dfc.drop(['vecis'], axis= 1)
#toronto_data = pd.merge(torn_data, neights1, left_index=True, right_index=True)
toronto_data.head()

,PostalCode,Borough,Postal Code,Latitude,Longitude
0,M5H,Downtown Toronto,M5H,43.650571,-79.384568
1,M1S,Scarborough,M1S,43.794200,-79.262029
2,M1V,Scarborough,M1V,43.815252,-79.284577
3,M9V,Etobicoke,M9V,43.739416,-79.588437
4,M8W,Etobicoke,M8W,43.602414,-79.543484


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('vecis', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=5).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 1, 0, 0, 0, 4, 0, 4, 4])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [50]:
#add clustering labels
neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhood_venues_sorted.set_index('vecis'), on='vecis')

#toronto_merged['Clusteres'] = toronto_merged['Cluster Labels'].astype('int')
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(2).astype(int)

toronto_merged.head() # check the last columns!


,PostalCode,Borough,vecis,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5H,Downtown Toronto,"Adelaide,King,Richmond",M5H,43.650571,-79.384568,4,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Cosmetics Shop,Hotel,American Restaurant,Burger Joint,Restaurant
1,M1S,Scarborough,Agincourt,M1S,43.794200,-79.262029,4,Breakfast Spot,Sandwich Place,Lounge,Skating Rink,Yoga Studio,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",M1V,43.815252,-79.284577,1,Playground,Park,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",M9V,43.739416,-79.588437,0,Grocery Store,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Beer Store,Coffee Shop,Pharmacy,Empanada Restaurant,Ethiopian Restaurant
4,M8W,Etobicoke,"Alderwood,Long Branch",M8W,43.602414,-79.543484,0,Pizza Place,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pool,Gym,Pharmacy,Empanada Restaurant,Ethiopian Restaurant


Finally, let's visualize the resulting clusters

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['vecis'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,PostalCode,Borough,vecis,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",M9V,43.739416,-79.588437,0,Grocery Store,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Beer Store,Coffee Shop,Pharmacy,Empanada Restaurant,Ethiopian Restaurant
4,M8W,Etobicoke,"Alderwood,Long Branch",M8W,43.602414,-79.543484,0,Pizza Place,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pool,Gym,Pharmacy,Empanada Restaurant,Ethiopian Restaurant
5,M3H,North York,"Bathurst Manor,Downsview North,Wilson Heights",M3H,43.754328,-79.442259,0,Coffee Shop,Pizza Place,Deli / Bodega,Fast Food Restaurant,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Supermarket,Restaurant,Sushi Restaurant
7,M5M,North York,"Bedford Park,Lawrence Manor East",M5M,43.733283,-79.419750,0,Pizza Place,Coffee Shop,Italian Restaurant,Pharmacy,Thai Restaurant,Comfort Food Restaurant,Pub,Restaurant,Café,Butcher
10,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",M9C,43.643515,-79.577201,0,Pharmacy,Liquor Store,Café,Shopping Plaza,Beer Store,Coffee Shop,Convenience Store,Pizza Place,Falafel Restaurant,Farmers Market
24,M1T,Scarborough,"Clarks Corners,Sullivan,Tam O'Shanter",M1T,43.781638,-79.304302,0,Pizza Place,Noodle House,Shopping Mall,Fried Chicken Joint,Bank,Italian Restaurant,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Fast Food Restaurant
26,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",M9B,43.650943,-79.554724,0,Bank,Gift Shop,Yoga Studio,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
30,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",M4V,43.686412,-79.400049,0,Coffee Shop,Pub,Pizza Place,Bagel Shop,Restaurant,Liquor Store,Sports Bar,Supermarket,Sushi Restaurant,Fried Chicken Joint
46,M5P,Central Toronto,"Forest Hill North,Forest Hill West",M5P,43.696948,-79.411307,0,Jewelry Store,Sushi Restaurant,Trail,Bus Line,Yoga Studio,Doner Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
48,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711,0,Pizza Place,Rental Car Location,Breakfast Spot,Medical Center,Mexican Restaurant,Electronics Store,Intersection,Creperie,Donut Shop,Concert Hall


#### Cluster 2

In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,PostalCode,Borough,vecis,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",M1V,43.815252,-79.284577,1,Playground,Park,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
13,M3K,North York,"CFB Toronto,Downsview East",M3K,43.737473,-79.464763,1,Playground,Airport,Park,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
16,M6E,York,Caledonia-Fairbanks,M6E,43.689026,-79.453512,1,Park,Women's Store,Fast Food Restaurant,Market,Hakka Restaurant,Field,Harbor / Marina,Farmers Market,Falafel Restaurant,Event Space
41,M4J,East York,East Toronto,M4J,43.685347,-79.338106,1,Park,Convenience Store,Rental Car Location,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
60,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",M9R,43.688905,-79.554724,1,Pizza Place,Park,Bus Line,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
64,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,1,Park,Swim School,Bus Line,Yoga Studio,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
75,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,1,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
76,M4W,Downtown Toronto,Rosedale,M4W,43.679563,-79.377529,1,Park,Building,Trail,Playground,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
96,M9N,York,Weston,M9N,43.706876,-79.518188,1,Park,Convenience Store,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
102,M2P,North York,York Mills West,M2P,43.752758,-79.400049,1,Park,Convenience Store,Bank,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant


#### Cluster 3

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,PostalCode,Borough,vecis,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,M4T,Central Toronto,"Moore Park,Summerhill East",M4T,43.689574,-79.383160,2,Playground,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Comfort Food Restaurant
69,M2M,North York,"Newtonbrook,Willowdale",M2M,43.789053,-79.408493,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476,2,Playground,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Comfort Food Restaurant
93,M1X,Scarborough,Upper Rouge,M1X,43.836125,-79.205636,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 4

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,PostalCode,Borough,vecis,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353,3,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space


#### Cluster 5

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,PostalCode,Borough,vecis,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5H,Downtown Toronto,"Adelaide,King,Richmond",M5H,43.650571,-79.384568,4,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Cosmetics Shop,Hotel,American Restaurant,Burger Joint,Restaurant
1,M1S,Scarborough,Agincourt,M1S,43.794200,-79.262029,4,Breakfast Spot,Sandwich Place,Lounge,Skating Rink,Yoga Studio,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
6,M2K,North York,Bayview Village,M2K,43.786947,-79.385975,4,Japanese Restaurant,Chinese Restaurant,Bank,Café,Yoga Studio,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
8,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,4,Coffee Shop,Cocktail Bar,Bakery,Café,Cheese Shop,Seafood Restaurant,Farmers Market,Steakhouse,Beer Bar,Irish Pub
9,M1N,Scarborough,"Birch Cliff,Cliffside West",M1N,43.692657,-79.264848,4,Café,College Stadium,General Entertainment,Skating Rink,Yoga Studio,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
11,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",M6K,43.636847,-79.428191,4,Café,Breakfast Spot,Coffee Shop,Restaurant,Caribbean Restaurant,Bar,Italian Restaurant,Stadium,Climbing Gym,Intersection
12,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,M7Y,43.662744,-79.321558,4,Light Rail Station,Pizza Place,Garden Center,Recording Studio,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery,Farmers Market
14,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",M5V,43.628947,-79.394420,4,Airport Service,Airport Lounge,Airport Terminal,Plane,Boutique,Coffee Shop,Sculpture Garden,Boat or Ferry,Bar,Harbor / Marina
15,M4X,Downtown Toronto,"Cabbagetown,St. James Town",M4X,43.667967,-79.367675,4,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Pet Store,Pizza Place,Pub,Chinese Restaurant,Café,Market
17,M7R,Mississauga,Canada Post Gateway Processing Centre,M7R,43.636966,-79.615819,4,Hotel,Coffee Shop,Mediterranean Restaurant,Gym / Fitness Center,Fried Chicken Joint,Sandwich Place,Burrito Place,American Restaurant,Middle Eastern Restaurant,Yoga Studio
